In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.0008
    elif epoch<30:
        return 0.0004
    else:
        return 0.0001
    

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=8)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.63885, saving model to best_m.h5
 - 2s - loss: 0.6831 - acc: 0.5795 - val_loss: 0.6388 - val_acc: 0.5771
Epoch 2/120
Epoch 00002: val_loss improved from 0.63885 to 0.55763, saving model to best_m.h5
 - 2s - loss: 0.6140 - acc: 0.6473 - val_loss: 0.5576 - val_acc: 0.7910
Epoch 3/120
Epoch 00003: val_loss improved from 0.55763 to 0.45450, saving model to best_m.h5
 - 2s - loss: 0.5494 - acc: 0.7206 - val_loss: 0.4545 - val_acc: 0.8458
Epoch 4/120
Epoch 00004: val_loss improved from 0.45450 to 0.42747, saving model to best_m.h5
 - 2s - loss: 0.4922 - acc: 0.7692 - val_loss: 0.4275 - val_acc: 0.8706
Epoch 5/120
Epoch 00005: val_loss improved from 0.42747 to 0.35438, saving model to best_m.h5
 - 2s - loss: 0.4579 - acc: 0.7777 - val_loss: 0.3544 - val_acc: 0.8308
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.4576 - acc: 0.7853 - val_loss: 0.4000 - val_acc: 0.8507
Epoch 7/120
Epoch 00007: val_loss improved from 0.35

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.2160 - acc: 0.9092 - val_loss: 0.1841 - val_acc: 0.9104
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2068 - acc: 0.9124 - val_loss: 0.1885 - val_acc: 0.9055
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2257 - acc: 0.8987 - val_loss: 0.1900 - val_acc: 0.9104
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.2154 - acc: 0.9131 - val_loss: 0.1870 - val_acc: 0.9104
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.2241 - acc: 0.9009 - val_loss: 0.1832 - val_acc: 0.9204
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2116 - acc: 0.9045 - val_loss: 0.1799 - val_acc: 0.9104
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2359 - acc: 0.8989 - val_loss: 0.1878 - val_acc: 0.9104
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2279 - acc: 0.9073 - val_loss: 0.1889 - val_acc: 0.9055
Epoch 70/120
Epo

Epoch 7/120
Epoch 00007: val_loss improved from 0.38988 to 0.37700, saving model to best_m.h5
 - 2s - loss: 0.4210 - acc: 0.7972 - val_loss: 0.3770 - val_acc: 0.8209
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.4211 - acc: 0.8115 - val_loss: 0.4505 - val_acc: 0.7711
Epoch 9/120
Epoch 00009: val_loss improved from 0.37700 to 0.30494, saving model to best_m.h5
 - 2s - loss: 0.4225 - acc: 0.8136 - val_loss: 0.3049 - val_acc: 0.8657
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.3872 - acc: 0.8150 - val_loss: 0.3163 - val_acc: 0.8408
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.3553 - acc: 0.8337 - val_loss: 0.3253 - val_acc: 0.8358
Epoch 12/120
Epoch 00012: val_loss improved from 0.30494 to 0.28107, saving model to best_m.h5
 - 2s - loss: 0.3488 - acc: 0.8416 - val_loss: 0.2811 - val_acc: 0.8856
Epoch 13/120
Epoch 00013: val_loss improved from 0.28107 to 0.26426, saving model to best_m.h5
 - 2s - loss: 0.3448 - acc: 0.8427 - v

Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2555 - acc: 0.8854 - val_loss: 0.1938 - val_acc: 0.9154
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2303 - acc: 0.8977 - val_loss: 0.1790 - val_acc: 0.9254
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.2357 - acc: 0.8988 - val_loss: 0.1877 - val_acc: 0.9204
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.2250 - acc: 0.8937 - val_loss: 0.1779 - val_acc: 0.9254
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.2268 - acc: 0.9030 - val_loss: 0.2016 - val_acc: 0.9154
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.2382 - acc: 0.8902 - val_loss: 0.2119 - val_acc: 0.9055
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.2485 - acc: 0.8898 - val_loss: 0.1971 - val_acc: 0.9204
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.2311 - acc: 0.8869 - val_loss: 0.1804 - val_acc: 0.9254
Epoch 76/120
Epo

Epoch 11/120
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.3191 - acc: 0.8528 - val_loss: 0.3130 - val_acc: 0.8756
Epoch 12/120
Epoch 00012: val_loss improved from 0.30569 to 0.27355, saving model to best_m.h5
 - 2s - loss: 0.2955 - acc: 0.8743 - val_loss: 0.2736 - val_acc: 0.8955
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2846 - acc: 0.8683 - val_loss: 0.3016 - val_acc: 0.8806
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2724 - acc: 0.8851 - val_loss: 0.3118 - val_acc: 0.8756
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 2s - loss: 0.3010 - acc: 0.8589 - val_loss: 0.2778 - val_acc: 0.8756
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2686 - acc: 0.8700 - val_loss: 0.3068 - val_acc: 0.8706
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.3035 - acc: 0.8621 - val_loss: 0.3051 - val_acc: 0.8607
Epoch 18/120
Epoch 00018: val_loss did not improve
 - 2s - loss: 0.2668 - acc: 0.8848 - val_los

...

Epoch 95/120
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.1967 - acc: 0.9152 - val_loss: 0.1764 - val_acc: 0.9400
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1821 - acc: 0.9179 - val_loss: 0.1840 - val_acc: 0.9250
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.2055 - acc: 0.9198 - val_loss: 0.1823 - val_acc: 0.9300
Epoch 98/120
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.2057 - acc: 0.9138 - val_loss: 0.1981 - val_acc: 0.9200
Epoch 99/120
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.2147 - acc: 0.9116 - val_loss: 0.2012 - val_acc: 0.9150
Epoch 100/120
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.2054 - acc: 0.9049 - val_loss: 0.1870 - val_acc: 0.9200
Epoch 101/120
Epoch 00101: val_loss did not improve
 - 2s - loss: 0.2029 - acc: 0.9071 - val_loss: 0.1834 - val_acc: 0.9350
Epoch 102/120
Epoch 00102: val_loss did not improve
 - 2s - loss: 0.1984 - acc: 0.9152 - val_loss: 0.1752 - val_acc: 0.9450
Epoch 103/120

Epoch 37/120
Epoch 00037: val_loss did not improve
 - 2s - loss: 0.2473 - acc: 0.8968 - val_loss: 0.2737 - val_acc: 0.8900
Epoch 38/120
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.2462 - acc: 0.8843 - val_loss: 0.2522 - val_acc: 0.8950
Epoch 39/120
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.2495 - acc: 0.8910 - val_loss: 0.2579 - val_acc: 0.8900
Epoch 40/120
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.2423 - acc: 0.8973 - val_loss: 0.2799 - val_acc: 0.8800
Epoch 41/120
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.2492 - acc: 0.8951 - val_loss: 0.2573 - val_acc: 0.8800
Epoch 42/120
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.2250 - acc: 0.9049 - val_loss: 0.2632 - val_acc: 0.8950
Epoch 43/120
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.2496 - acc: 0.8934 - val_loss: 0.2565 - val_acc: 0.8950
Epoch 44/120
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.2295 - acc: 0.8999 - val_loss: 0.2810 - val_acc: 0.8800
Epoch 45/120
Epo

Epoch 102/120
Epoch 00102: val_loss did not improve
 - 2s - loss: 0.2256 - acc: 0.9124 - val_loss: 0.2791 - val_acc: 0.8850
Epoch 103/120
Epoch 00103: val_loss did not improve
 - 2s - loss: 0.2056 - acc: 0.9066 - val_loss: 0.2440 - val_acc: 0.8950
Epoch 104/120
Epoch 00104: val_loss did not improve
 - 2s - loss: 0.2137 - acc: 0.9066 - val_loss: 0.2524 - val_acc: 0.8900
Epoch 105/120
Epoch 00105: val_loss improved from 0.22843 to 0.22741, saving model to best_m.h5
 - 2s - loss: 0.2106 - acc: 0.9085 - val_loss: 0.2274 - val_acc: 0.9050
Epoch 106/120
Epoch 00106: val_loss did not improve
 - 2s - loss: 0.1971 - acc: 0.9150 - val_loss: 0.2659 - val_acc: 0.8850
Epoch 107/120
Epoch 00107: val_loss improved from 0.22741 to 0.22728, saving model to best_m.h5
 - 2s - loss: 0.2252 - acc: 0.9073 - val_loss: 0.2273 - val_acc: 0.9050
Epoch 108/120
Epoch 00108: val_loss improved from 0.22728 to 0.21765, saving model to best_m.h5
 - 2s - loss: 0.2174 - acc: 0.9083 - val_loss: 0.2177 - val_acc: 0.9100


Epoch 44/120
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.2173 - acc: 0.9078 - val_loss: 0.2768 - val_acc: 0.8850
Epoch 45/120
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.2243 - acc: 0.8934 - val_loss: 0.2978 - val_acc: 0.8800
Epoch 46/120
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.2086 - acc: 0.9181 - val_loss: 0.3034 - val_acc: 0.8650
Epoch 47/120
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.2171 - acc: 0.9143 - val_loss: 0.2822 - val_acc: 0.8950
Epoch 48/120
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.2452 - acc: 0.8898 - val_loss: 0.3179 - val_acc: 0.8900
Epoch 49/120
Epoch 00049: val_loss did not improve
 - 2s - loss: 0.2096 - acc: 0.9073 - val_loss: 0.2836 - val_acc: 0.8800
Epoch 50/120
Epoch 00050: val_loss did not improve
 - 2s - loss: 0.2229 - acc: 0.9016 - val_loss: 0.3092 - val_acc: 0.8900
Epoch 51/120
Epoch 00051: val_loss did not improve
 - 2s - loss: 0.2300 - acc: 0.9030 - val_loss: 0.3085 - val_acc: 0.8900
Epoch 52/120
Epo

Epoch 110/120
Epoch 00110: val_loss did not improve
 - 2s - loss: 0.1898 - acc: 0.9157 - val_loss: 0.2887 - val_acc: 0.9050
Epoch 111/120
Epoch 00111: val_loss did not improve
 - 2s - loss: 0.1967 - acc: 0.9243 - val_loss: 0.2987 - val_acc: 0.8950
Epoch 112/120
Epoch 00112: val_loss did not improve
 - 2s - loss: 0.2049 - acc: 0.9128 - val_loss: 0.2831 - val_acc: 0.8900
Epoch 113/120
Epoch 00113: val_loss did not improve
 - 2s - loss: 0.1972 - acc: 0.9080 - val_loss: 0.2749 - val_acc: 0.8950
Epoch 114/120
Epoch 00114: val_loss did not improve
 - 2s - loss: 0.1975 - acc: 0.9124 - val_loss: 0.2857 - val_acc: 0.8900
Epoch 115/120
Epoch 00115: val_loss did not improve
 - 2s - loss: 0.1919 - acc: 0.9186 - val_loss: 0.2790 - val_acc: 0.8950
Epoch 116/120
Epoch 00116: val_loss did not improve
 - 2s - loss: 0.1919 - acc: 0.9150 - val_loss: 0.2929 - val_acc: 0.8850
Epoch 117/120
Epoch 00117: val_loss did not improve
 - 2s - loss: 0.1963 - acc: 0.9076 - val_loss: 0.3085 - val_acc: 0.8750
Epoch 11

In [6]:
import pickle
with open('../features/cnn_1_aug_1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

#pre 3219
# new 2965

# skimage 2725

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug_1_fold8.csv', index=False)

0.197575578605
         id  is_iceberg
0  5941774d    0.103387
1  4023181e    0.985510
2  b20200e4    0.918696
3  e7f018bb    0.995533
4  4371c8c3    0.832899


In [7]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [8]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=8)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.59886, saving model to best_m.h5
 - 2s - loss: 0.6666 - acc: 0.5795 - val_loss: 0.5989 - val_acc: 0.5970
Epoch 2/120
Epoch 00002: val_loss improved from 0.59886 to 0.34920, saving model to best_m.h5
 - 1s - loss: 0.5143 - acc: 0.7493 - val_loss: 0.3492 - val_acc: 0.8458
Epoch 3/120
Epoch 00003: val_loss improved from 0.34920 to 0.33103, saving model to best_m.h5
 - 1s - loss: 0.4389 - acc: 0.8054 - val_loss: 0.3310 - val_acc: 0.8706
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.3753 - acc: 0.8280 - val_loss: 0.3745 - val_acc: 0.8458
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.4289 - acc: 0.8004 - val_loss: 0.3504 - val_acc: 0.8458
Epoch 6/120
Epoch 00006: val_loss improved from 0.33103 to 0.29485, saving model to best_m.h5
 - 1s - loss: 0.3391 - acc: 0.8524 - val_loss: 0.2949 - val_acc: 0.8507
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.3656 - acc: 0.8359 - val_loss:

Epoch 63/120
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1798 - acc: 0.9257 - val_loss: 0.1640 - val_acc: 0.9204
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1905 - acc: 0.9160 - val_loss: 0.1599 - val_acc: 0.9353
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1661 - acc: 0.9282 - val_loss: 0.1846 - val_acc: 0.9204
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1763 - acc: 0.9195 - val_loss: 0.1724 - val_acc: 0.9154
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1977 - acc: 0.9156 - val_loss: 0.1585 - val_acc: 0.9403
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1818 - acc: 0.9242 - val_loss: 0.1673 - val_acc: 0.9254
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1869 - acc: 0.9181 - val_loss: 0.1648 - val_acc: 0.9353
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1726 - acc: 0.9246 - val_loss: 0.1611 - val_acc: 0.9303
Epoch 71/120
Epo

Epoch 8/120
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.3262 - acc: 0.8636 - val_loss: 0.3878 - val_acc: 0.8060
Epoch 9/120
Epoch 00009: val_loss improved from 0.28750 to 0.23332, saving model to best_m.h5
 - 1s - loss: 0.3141 - acc: 0.8642 - val_loss: 0.2333 - val_acc: 0.8756
Epoch 10/120
Epoch 00010: val_loss improved from 0.23332 to 0.21721, saving model to best_m.h5
 - 1s - loss: 0.3110 - acc: 0.8646 - val_loss: 0.2172 - val_acc: 0.9055
Epoch 11/120
Epoch 00011: val_loss improved from 0.21721 to 0.18011, saving model to best_m.h5
 - 1s - loss: 0.2781 - acc: 0.8755 - val_loss: 0.1801 - val_acc: 0.9154
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.2682 - acc: 0.8858 - val_loss: 0.2314 - val_acc: 0.8955
Epoch 13/120
Epoch 00013: val_loss improved from 0.18011 to 0.17917, saving model to best_m.h5
 - 1s - loss: 0.2649 - acc: 0.8941 - val_loss: 0.1792 - val_acc: 0.9104
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.2568 - acc: 0.8880 - 

Epoch 70/120
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1794 - acc: 0.9178 - val_loss: 0.1728 - val_acc: 0.9154
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.1820 - acc: 0.9253 - val_loss: 0.1585 - val_acc: 0.9204
Epoch 72/120
Epoch 00072: val_loss improved from 0.13322 to 0.13294, saving model to best_m.h5
 - 1s - loss: 0.1960 - acc: 0.9242 - val_loss: 0.1329 - val_acc: 0.9502
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.1717 - acc: 0.9274 - val_loss: 0.1609 - val_acc: 0.9104
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.1902 - acc: 0.9157 - val_loss: 0.1373 - val_acc: 0.9403
Epoch 75/120
Epoch 00075: val_loss improved from 0.13294 to 0.12087, saving model to best_m.h5
 - 1s - loss: 0.1891 - acc: 0.9210 - val_loss: 0.1209 - val_acc: 0.9453
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.2024 - acc: 0.9152 - val_loss: 0.1530 - val_acc: 0.9254
Epoch 77/120
Epoch 00077: val_loss did not improve


Epoch 13/120
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.2552 - acc: 0.8922 - val_loss: 0.3125 - val_acc: 0.8657
Epoch 14/120
Epoch 00014: val_loss improved from 0.30214 to 0.29874, saving model to best_m.h5
 - 1s - loss: 0.2762 - acc: 0.8837 - val_loss: 0.2987 - val_acc: 0.8955
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2468 - acc: 0.8951 - val_loss: 0.3012 - val_acc: 0.8706
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 1s - loss: 0.2605 - acc: 0.8899 - val_loss: 0.3073 - val_acc: 0.8706
Epoch 17/120
Epoch 00017: val_loss improved from 0.29874 to 0.28897, saving model to best_m.h5
 - 1s - loss: 0.2561 - acc: 0.8984 - val_loss: 0.2890 - val_acc: 0.8955
Epoch 18/120
Epoch 00018: val_loss improved from 0.28897 to 0.27842, saving model to best_m.h5
 - 1s - loss: 0.2383 - acc: 0.9048 - val_loss: 0.2784 - val_acc: 0.8856
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.2353 - acc: 0.8998 - val_loss: 0.3019 - val_acc: 0.8756
Epoch 2

...

Epoch 99/120
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.1806 - acc: 0.9298 - val_loss: 0.2304 - val_acc: 0.9150
Epoch 100/120
Epoch 00100: val_loss did not improve
 - 1s - loss: 0.1934 - acc: 0.9179 - val_loss: 0.2390 - val_acc: 0.9150
Epoch 101/120
Epoch 00101: val_loss did not improve
 - 1s - loss: 0.1689 - acc: 0.9246 - val_loss: 0.2357 - val_acc: 0.9150
Epoch 102/120
Epoch 00102: val_loss did not improve
 - 1s - loss: 0.1821 - acc: 0.9193 - val_loss: 0.2178 - val_acc: 0.9200
Epoch 103/120
Epoch 00103: val_loss improved from 0.21600 to 0.21355, saving model to best_m.h5
 - 1s - loss: 0.1686 - acc: 0.9318 - val_loss: 0.2135 - val_acc: 0.9150
Epoch 104/120
Epoch 00104: val_loss did not improve
 - 1s - loss: 0.1730 - acc: 0.9260 - val_loss: 0.2283 - val_acc: 0.9250
Epoch 105/120
Epoch 00105: val_loss did not improve
 - 1s - loss: 0.1801 - acc: 0.9186 - val_loss: 0.2192 - val_acc: 0.9050
Epoch 106/120
Epoch 00106: val_loss did not improve
 - 1s - loss: 0.1843 - acc: 0.9248 - 

Epoch 41/120
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.1692 - acc: 0.9397 - val_loss: 0.2281 - val_acc: 0.8900
Epoch 42/120
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.2004 - acc: 0.9188 - val_loss: 0.2177 - val_acc: 0.9200
Epoch 43/120
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.1845 - acc: 0.9303 - val_loss: 0.2153 - val_acc: 0.9200
Epoch 44/120
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.1777 - acc: 0.9253 - val_loss: 0.2216 - val_acc: 0.9150
Epoch 45/120
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.1854 - acc: 0.9210 - val_loss: 0.2259 - val_acc: 0.9050
Epoch 46/120
Epoch 00046: val_loss did not improve
 - 1s - loss: 0.1751 - acc: 0.9286 - val_loss: 0.2385 - val_acc: 0.8850
Epoch 47/120
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.2004 - acc: 0.9188 - val_loss: 0.2213 - val_acc: 0.8950
Epoch 48/120
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.1749 - acc: 0.9262 - val_loss: 0.2248 - val_acc: 0.9100
Epoch 49/120
Epo

Epoch 108/120
Epoch 00108: val_loss did not improve
 - 1s - loss: 0.1507 - acc: 0.9361 - val_loss: 0.2605 - val_acc: 0.8800
Epoch 109/120
Epoch 00109: val_loss did not improve
 - 1s - loss: 0.1497 - acc: 0.9461 - val_loss: 0.2363 - val_acc: 0.8950
Epoch 110/120
Epoch 00110: val_loss did not improve
 - 1s - loss: 0.1468 - acc: 0.9447 - val_loss: 0.2523 - val_acc: 0.9000
Epoch 111/120
Epoch 00111: val_loss did not improve
 - 1s - loss: 0.1396 - acc: 0.9468 - val_loss: 0.2396 - val_acc: 0.8950
Epoch 112/120
Epoch 00112: val_loss did not improve
 - 1s - loss: 0.1603 - acc: 0.9361 - val_loss: 0.2340 - val_acc: 0.9050
Epoch 113/120
Epoch 00113: val_loss did not improve
 - 1s - loss: 0.1564 - acc: 0.9375 - val_loss: 0.2287 - val_acc: 0.9050
Epoch 114/120
Epoch 00114: val_loss did not improve
 - 1s - loss: 0.1355 - acc: 0.9440 - val_loss: 0.2319 - val_acc: 0.9000
Epoch 115/120
Epoch 00115: val_loss did not improve
 - 1s - loss: 0.1370 - acc: 0.9382 - val_loss: 0.2339 - val_acc: 0.9000
Epoch 11

Epoch 52/120
Epoch 00052: val_loss did not improve
 - 1s - loss: 0.1891 - acc: 0.9181 - val_loss: 0.3371 - val_acc: 0.9000
Epoch 53/120
Epoch 00053: val_loss did not improve
 - 1s - loss: 0.1788 - acc: 0.9262 - val_loss: 0.3265 - val_acc: 0.8950
Epoch 54/120
Epoch 00054: val_loss did not improve
 - 1s - loss: 0.1724 - acc: 0.9246 - val_loss: 0.3402 - val_acc: 0.8900
Epoch 55/120
Epoch 00055: val_loss did not improve
 - 1s - loss: 0.1954 - acc: 0.9088 - val_loss: 0.3199 - val_acc: 0.9050
Epoch 56/120
Epoch 00056: val_loss did not improve
 - 1s - loss: 0.1811 - acc: 0.9282 - val_loss: 0.3232 - val_acc: 0.8800
Epoch 57/120
Epoch 00057: val_loss improved from 0.30821 to 0.30806, saving model to best_m.h5
 - 1s - loss: 0.1819 - acc: 0.9246 - val_loss: 0.3081 - val_acc: 0.9100
Epoch 58/120
Epoch 00058: val_loss did not improve
 - 1s - loss: 0.1833 - acc: 0.9274 - val_loss: 0.3283 - val_acc: 0.8950
Epoch 59/120
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.1699 - acc: 0.9186 - val_los

Epoch 118/120
Epoch 00118: val_loss did not improve
 - 1s - loss: 0.1426 - acc: 0.9423 - val_loss: 0.3508 - val_acc: 0.9100
Epoch 119/120
Epoch 00119: val_loss did not improve
 - 1s - loss: 0.1445 - acc: 0.9385 - val_loss: 0.3396 - val_acc: 0.8900
Epoch 120/120
Epoch 00120: val_loss did not improve
 - 1s - loss: 0.1508 - acc: 0.9361 - val_loss: 0.3524 - val_acc: 0.8900


In [9]:
with open('../features/cnn_2_aug_1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2728
# skimage 2710

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug_1_fold8.csv', index=False)

0.197748331302
         id  is_iceberg
0  5941774d    0.073844
1  4023181e    0.795579
2  b20200e4    0.420294
3  e7f018bb    0.998610
4  4371c8c3    0.081664


In [10]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [11]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.65841, saving model to best_m.h5
 - 4s - loss: 0.6958 - acc: 0.5302 - val_loss: 0.6584 - val_acc: 0.5016
Epoch 2/120
Epoch 00002: val_loss improved from 0.65841 to 0.55948, saving model to best_m.h5
 - 2s - loss: 0.6011 - acc: 0.6649 - val_loss: 0.5595 - val_acc: 0.7165
Epoch 3/120
Epoch 00003: val_loss improved from 0.55948 to 0.41684, saving model to best_m.h5
 - 2s - loss: 0.5334 - acc: 0.7443 - val_loss: 0.4168 - val_acc: 0.8567
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.4824 - acc: 0.7777 - val_loss: 0.5936 - val_acc: 0.6698
Epoch 5/120
Epoch 00005: val_loss improved from 0.41684 to 0.39750, saving model to best_m.h5
 - 2s - loss: 0.4897 - acc: 0.7703 - val_loss: 0.3975 - val_acc: 0.8536
Epoch 6/120
Epoch 00006: val_loss improved from 0.39750 to 0.36294, saving model to best_m.h5
 - 2s - loss: 0.4260 - acc: 0.7927 - val_loss: 0.3629 - val_acc: 0.8847
Epoch 7/120
Epoch 00007: val_loss did not improve
 -

Epoch 64/120
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2115 - acc: 0.9125 - val_loss: 0.1965 - val_acc: 0.9065
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.2381 - acc: 0.9000 - val_loss: 0.1998 - val_acc: 0.9097
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.2192 - acc: 0.9148 - val_loss: 0.1992 - val_acc: 0.9065
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2190 - acc: 0.9055 - val_loss: 0.1893 - val_acc: 0.9159
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2128 - acc: 0.9148 - val_loss: 0.1894 - val_acc: 0.9034
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2351 - acc: 0.8959 - val_loss: 0.2073 - val_acc: 0.9159
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.2121 - acc: 0.9078 - val_loss: 0.1967 - val_acc: 0.9128
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.2370 - acc: 0.8933 - val_loss: 0.1986 - val_acc: 0.9190
Epoch 72/120
Epo

Epoch 8/120
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.3920 - acc: 0.8255 - val_loss: 0.3871 - val_acc: 0.8474
Epoch 9/120
Epoch 00009: val_loss improved from 0.36182 to 0.33848, saving model to best_m.h5
 - 2s - loss: 0.3650 - acc: 0.8312 - val_loss: 0.3385 - val_acc: 0.8505
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.3371 - acc: 0.8422 - val_loss: 0.4006 - val_acc: 0.8069
Epoch 11/120
Epoch 00011: val_loss improved from 0.33848 to 0.31248, saving model to best_m.h5
 - 2s - loss: 0.3123 - acc: 0.8765 - val_loss: 0.3125 - val_acc: 0.8567
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 2s - loss: 0.2921 - acc: 0.8695 - val_loss: 0.3305 - val_acc: 0.8567
Epoch 13/120
Epoch 00013: val_loss improved from 0.31248 to 0.29969, saving model to best_m.h5
 - 2s - loss: 0.2858 - acc: 0.8844 - val_loss: 0.2997 - val_acc: 0.8723
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2941 - acc: 0.8781 - val_loss: 0.3031 - val_acc: 0.8754
Epoch 15/

Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.2071 - acc: 0.9117 - val_loss: 0.2292 - val_acc: 0.9097
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.2182 - acc: 0.9070 - val_loss: 0.2287 - val_acc: 0.9034
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.2006 - acc: 0.9164 - val_loss: 0.2387 - val_acc: 0.8879
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1920 - acc: 0.9203 - val_loss: 0.2341 - val_acc: 0.9097
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.2202 - acc: 0.9086 - val_loss: 0.2222 - val_acc: 0.9190
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.1876 - acc: 0.9242 - val_loss: 0.2217 - val_acc: 0.9128
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.2077 - acc: 0.9086 - val_loss: 0.2383 - val_acc: 0.9034
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.2069 - acc: 0.9076 - val_loss: 0.2217 - val_acc: 0.9159
Epoch 78/120
Epo

Epoch 00012: val_loss improved from 0.22795 to 0.22364, saving model to best_m.h5
 - 2s - loss: 0.3212 - acc: 0.8539 - val_loss: 0.2236 - val_acc: 0.9065
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.3122 - acc: 0.8599 - val_loss: 0.2699 - val_acc: 0.8879
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2700 - acc: 0.8797 - val_loss: 0.2242 - val_acc: 0.9065
Epoch 15/120
Epoch 00015: val_loss improved from 0.22364 to 0.21764, saving model to best_m.h5
 - 2s - loss: 0.3157 - acc: 0.8539 - val_loss: 0.2176 - val_acc: 0.9097
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.3180 - acc: 0.8687 - val_loss: 0.2273 - val_acc: 0.9097
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2976 - acc: 0.8672 - val_loss: 0.2667 - val_acc: 0.8816
Epoch 18/120
Epoch 00018: val_loss did not improve
 - 2s - loss: 0.2899 - acc: 0.8640 - val_loss: 0.2379 - val_acc: 0.8879
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 2s - loss:

Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.2165 - acc: 0.9000 - val_loss: 0.1983 - val_acc: 0.9346
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.2178 - acc: 0.9055 - val_loss: 0.1968 - val_acc: 0.9346
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.2154 - acc: 0.9117 - val_loss: 0.2089 - val_acc: 0.9159
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.2204 - acc: 0.8976 - val_loss: 0.2155 - val_acc: 0.9252
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.2028 - acc: 0.9133 - val_loss: 0.2022 - val_acc: 0.9252
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 2s - loss: 0.2121 - acc: 0.9055 - val_loss: 0.2083 - val_acc: 0.9221
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 2s - loss: 0.2282 - acc: 0.9070 - val_loss: 0.1957 - val_acc: 0.9283
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 2s - loss: 0.2214 - acc: 0.9086 - val_loss: 0.1924 - val_acc: 0.9221
Epoch 84/120
Epo

Epoch 19/120
Epoch 00019: val_loss did not improve
 - 2s - loss: 0.2570 - acc: 0.8867 - val_loss: 0.2703 - val_acc: 0.8879
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.3062 - acc: 0.8693 - val_loss: 0.2857 - val_acc: 0.8754
Epoch 21/120
Epoch 00021: val_loss improved from 0.26181 to 0.25522, saving model to best_m.h5
 - 2s - loss: 0.2746 - acc: 0.8826 - val_loss: 0.2552 - val_acc: 0.8816
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.2415 - acc: 0.9008 - val_loss: 0.2622 - val_acc: 0.8879
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.2807 - acc: 0.8859 - val_loss: 0.2583 - val_acc: 0.8785
Epoch 24/120
Epoch 00024: val_loss improved from 0.25522 to 0.24573, saving model to best_m.h5
 - 2s - loss: 0.2734 - acc: 0.8797 - val_loss: 0.2457 - val_acc: 0.9065
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2649 - acc: 0.8844 - val_loss: 0.2700 - val_acc: 0.8847
Epoch 26/120
Epoch 00026: val_loss did not improve


Epoch 00083: val_loss did not improve
 - 2s - loss: 0.2032 - acc: 0.9180 - val_loss: 0.2422 - val_acc: 0.9097
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 2s - loss: 0.2090 - acc: 0.9219 - val_loss: 0.2351 - val_acc: 0.9097
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 2s - loss: 0.1992 - acc: 0.9156 - val_loss: 0.2647 - val_acc: 0.9003
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.2037 - acc: 0.9164 - val_loss: 0.2409 - val_acc: 0.9097
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.2261 - acc: 0.9068 - val_loss: 0.2498 - val_acc: 0.9034
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.2058 - acc: 0.9211 - val_loss: 0.2298 - val_acc: 0.9097
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.2055 - acc: 0.9164 - val_loss: 0.2312 - val_acc: 0.9128
Epoch 90/120
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.1965 - acc: 0.9180 - val_loss: 0.2279 - val_acc: 0.9221
Epoch 91/120
Epoch 00091: val

Epoch 27/120
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2557 - acc: 0.8930 - val_loss: 0.2847 - val_acc: 0.9000
Epoch 28/120
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2612 - acc: 0.8812 - val_loss: 0.2832 - val_acc: 0.9000
Epoch 29/120
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2455 - acc: 0.8930 - val_loss: 0.2977 - val_acc: 0.8906
Epoch 30/120
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2455 - acc: 0.8976 - val_loss: 0.2825 - val_acc: 0.9125
Epoch 31/120
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.2262 - acc: 0.8922 - val_loss: 0.2862 - val_acc: 0.8875
Epoch 32/120
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.2157 - acc: 0.9000 - val_loss: 0.2798 - val_acc: 0.8969
Epoch 33/120
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2427 - acc: 0.9024 - val_loss: 0.2791 - val_acc: 0.9031
Epoch 34/120
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.1960 - acc: 0.9101 - val_loss: 0.2900 - val_acc: 0.8969
Epoch 35/120
Epo

Epoch 92/120
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1944 - acc: 0.9125 - val_loss: 0.2941 - val_acc: 0.9000
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.1903 - acc: 0.9211 - val_loss: 0.2756 - val_acc: 0.9187
Epoch 94/120
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.1810 - acc: 0.9226 - val_loss: 0.2864 - val_acc: 0.9094
Epoch 95/120
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.1825 - acc: 0.9141 - val_loss: 0.3094 - val_acc: 0.8969
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1872 - acc: 0.9180 - val_loss: 0.2843 - val_acc: 0.9062
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.1926 - acc: 0.9187 - val_loss: 0.2989 - val_acc: 0.8969
Epoch 98/120
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.2015 - acc: 0.9180 - val_loss: 0.2703 - val_acc: 0.9062
Epoch 99/120
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.1991 - acc: 0.9118 - val_loss: 0.2683 - val_acc: 0.9000
Epoch 100/120
Ep

In [12]:
with open('../features/cnn_3_aug_1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug_1_fold8.csv', index=False)

0.2090692971
         id  is_iceberg
0  5941774d    0.165315
1  4023181e    0.660264
2  b20200e4    0.244674
3  e7f018bb    0.997191
4  4371c8c3    0.089288


In [13]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [14]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.84576, saving model to best_m.h5
 - 5s - loss: 0.6584 - acc: 0.7063 - val_loss: 0.8458 - val_acc: 0.5389
Epoch 2/120
Epoch 00002: val_loss improved from 0.84576 to 0.52221, saving model to best_m.h5
 - 2s - loss: 0.4117 - acc: 0.8013 - val_loss: 0.5222 - val_acc: 0.7072
Epoch 3/120
Epoch 00003: val_loss improved from 0.52221 to 0.46713, saving model to best_m.h5
 - 2s - loss: 0.3886 - acc: 0.8199 - val_loss: 0.4671 - val_acc: 0.7788
Epoch 4/120
Epoch 00004: val_loss improved from 0.46713 to 0.41262, saving model to best_m.h5
 - 2s - loss: 0.3656 - acc: 0.8406 - val_loss: 0.4126 - val_acc: 0.8224
Epoch 5/120
Epoch 00005: val_loss improved from 0.41262 to 0.38523, saving model to best_m.h5
 - 2s - loss: 0.3709 - acc: 0.8451 - val_loss: 0.3852 - val_acc: 0.8411
Epoch 6/120
Epoch 00006: val_loss improved from 0.38523 to 0.29617, saving model to best_m.h5
 - 2s - loss: 0.3301 - acc: 0.8513 - val_loss: 0.2962 - val_acc: 0.8879
Epoch 7/

Epoch 65/120
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1527 - acc: 0.9297 - val_loss: 0.2101 - val_acc: 0.9159
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1439 - acc: 0.9383 - val_loss: 0.2190 - val_acc: 0.9097
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1454 - acc: 0.9414 - val_loss: 0.2056 - val_acc: 0.9221
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1409 - acc: 0.9430 - val_loss: 0.2228 - val_acc: 0.9097
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1443 - acc: 0.9430 - val_loss: 0.2135 - val_acc: 0.9221
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1392 - acc: 0.9469 - val_loss: 0.3285 - val_acc: 0.8754
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.1421 - acc: 0.9406 - val_loss: 0.1995 - val_acc: 0.9190
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.1402 - acc: 0.9398 - val_loss: 0.2449 - val_acc: 0.8879
Epoch 73/120
Epo

Epoch 10/120
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.2773 - acc: 0.8719 - val_loss: 0.3062 - val_acc: 0.8660
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.2572 - acc: 0.8795 - val_loss: 0.2740 - val_acc: 0.8598
Epoch 12/120
Epoch 00012: val_loss improved from 0.25914 to 0.25344, saving model to best_m.h5
 - 2s - loss: 0.2474 - acc: 0.8937 - val_loss: 0.2534 - val_acc: 0.8785
Epoch 13/120
Epoch 00013: val_loss improved from 0.25344 to 0.22781, saving model to best_m.h5
 - 2s - loss: 0.2487 - acc: 0.8890 - val_loss: 0.2278 - val_acc: 0.8972
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2473 - acc: 0.8922 - val_loss: 0.2385 - val_acc: 0.8754
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 2s - loss: 0.2444 - acc: 0.8974 - val_loss: 0.3004 - val_acc: 0.8816
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2778 - acc: 0.8787 - val_loss: 0.3681 - val_acc: 0.8505
Epoch 17/120
Epoch 00017: val_loss did not improve


Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.1518 - acc: 0.9375 - val_loss: 0.2805 - val_acc: 0.9003
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.1413 - acc: 0.9422 - val_loss: 0.2489 - val_acc: 0.9034
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.1472 - acc: 0.9381 - val_loss: 0.2724 - val_acc: 0.9003
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.1380 - acc: 0.9469 - val_loss: 0.2600 - val_acc: 0.9159
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1399 - acc: 0.9492 - val_loss: 0.2890 - val_acc: 0.8972
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 2s - loss: 0.1461 - acc: 0.9414 - val_loss: 0.2458 - val_acc: 0.9034
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 2s - loss: 0.1283 - acc: 0.9474 - val_loss: 0.2558 - val_acc: 0.9034
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 2s - loss: 0.1318 - acc: 0.9506 - val_loss: 0.2618 - val_acc: 0.9034
Epoch 84/120
Epo

Epoch 20/120
Epoch 00020: val_loss improved from 0.19582 to 0.18595, saving model to best_m.h5
 - 2s - loss: 0.2425 - acc: 0.8976 - val_loss: 0.1860 - val_acc: 0.9252
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2551 - acc: 0.8922 - val_loss: 0.2079 - val_acc: 0.9034
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.2536 - acc: 0.8880 - val_loss: 0.2228 - val_acc: 0.8879
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.2545 - acc: 0.8906 - val_loss: 0.3418 - val_acc: 0.8442
Epoch 24/120
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.2274 - acc: 0.9109 - val_loss: 0.2381 - val_acc: 0.9003
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2361 - acc: 0.9016 - val_loss: 0.2478 - val_acc: 0.8847
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2248 - acc: 0.9115 - val_loss: 0.2307 - val_acc: 0.9003
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2452 - acc: 0.9021 - val_los

Epoch 86/120
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.1473 - acc: 0.9404 - val_loss: 0.2276 - val_acc: 0.9097
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.1569 - acc: 0.9388 - val_loss: 0.1924 - val_acc: 0.9221
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.1444 - acc: 0.9406 - val_loss: 0.1741 - val_acc: 0.9221
Epoch 89/120
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.1467 - acc: 0.9367 - val_loss: 0.1804 - val_acc: 0.9252
Epoch 90/120
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.1464 - acc: 0.9437 - val_loss: 0.1776 - val_acc: 0.9252
Epoch 91/120
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.1402 - acc: 0.9476 - val_loss: 0.1926 - val_acc: 0.9283
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1478 - acc: 0.9430 - val_loss: 0.1796 - val_acc: 0.9190
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.1257 - acc: 0.9523 - val_loss: 0.1857 - val_acc: 0.9252
Epoch 94/120
Epo

Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2367 - acc: 0.8984 - val_loss: 0.3333 - val_acc: 0.8598
Epoch 30/120
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2303 - acc: 0.9039 - val_loss: 0.2663 - val_acc: 0.8660
Epoch 31/120
Epoch 00031: val_loss improved from 0.19888 to 0.19883, saving model to best_m.h5
 - 2s - loss: 0.1827 - acc: 0.9258 - val_loss: 0.1988 - val_acc: 0.9128
Epoch 32/120
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.1960 - acc: 0.9185 - val_loss: 0.2017 - val_acc: 0.9190
Epoch 33/120
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.1615 - acc: 0.9328 - val_loss: 0.2007 - val_acc: 0.9065
Epoch 34/120
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.1678 - acc: 0.9312 - val_loss: 0.1989 - val_acc: 0.9034
Epoch 35/120
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.1957 - acc: 0.9232 - val_loss: 0.2044 - val_acc: 0.9315
Epoch 36/120
Epoch 00036: val_loss did not improve
 - 2s - loss: 0.1791 - acc: 0.9209 - val_loss: 0.2464 - v

Epoch 95/120
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.1298 - acc: 0.9484 - val_loss: 0.2544 - val_acc: 0.9003
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1337 - acc: 0.9476 - val_loss: 0.2767 - val_acc: 0.8910
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.1182 - acc: 0.9617 - val_loss: 0.2411 - val_acc: 0.9128
Epoch 98/120
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.1169 - acc: 0.9500 - val_loss: 0.2285 - val_acc: 0.9159
Epoch 99/120
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.1342 - acc: 0.9422 - val_loss: 0.2223 - val_acc: 0.9159
Epoch 100/120
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.1333 - acc: 0.9435 - val_loss: 0.2413 - val_acc: 0.9097
Epoch 101/120
Epoch 00101: val_loss did not improve
 - 2s - loss: 0.1243 - acc: 0.9406 - val_loss: 0.2446 - val_acc: 0.9065
Epoch 102/120
Epoch 00102: val_loss did not improve
 - 2s - loss: 0.1141 - acc: 0.9594 - val_loss: 0.2348 - val_acc: 0.9190
Epoch 103/120

Epoch 40/120
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.1454 - acc: 0.9398 - val_loss: 0.2998 - val_acc: 0.8906
Epoch 41/120
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.1566 - acc: 0.9320 - val_loss: 0.3138 - val_acc: 0.8656
Epoch 42/120
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.1437 - acc: 0.9453 - val_loss: 0.3168 - val_acc: 0.8750
Epoch 43/120
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.1566 - acc: 0.9352 - val_loss: 0.3254 - val_acc: 0.8781
Epoch 44/120
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1541 - acc: 0.9391 - val_loss: 0.3115 - val_acc: 0.8938
Epoch 45/120
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.1570 - acc: 0.9344 - val_loss: 0.3154 - val_acc: 0.8875
Epoch 46/120
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.1524 - acc: 0.9352 - val_loss: 0.3292 - val_acc: 0.8750
Epoch 47/120
Epoch 00047: val_loss did not improve
 - 2s - loss: 0.1771 - acc: 0.9172 - val_loss: 0.2917 - val_acc: 0.9031
Epoch 48/120
Epo

Epoch 107/120
Epoch 00107: val_loss did not improve
 - 2s - loss: 0.1150 - acc: 0.9492 - val_loss: 0.4961 - val_acc: 0.8375
Epoch 108/120
Epoch 00108: val_loss did not improve
 - 2s - loss: 0.1067 - acc: 0.9539 - val_loss: 0.3857 - val_acc: 0.8844
Epoch 109/120
Epoch 00109: val_loss did not improve
 - 2s - loss: 0.1147 - acc: 0.9555 - val_loss: 0.4980 - val_acc: 0.8469
Epoch 110/120
Epoch 00110: val_loss did not improve
 - 2s - loss: 0.1136 - acc: 0.9539 - val_loss: 0.4336 - val_acc: 0.8719
Epoch 111/120
Epoch 00111: val_loss did not improve
 - 2s - loss: 0.0920 - acc: 0.9648 - val_loss: 0.4355 - val_acc: 0.8781
Epoch 112/120
Epoch 00112: val_loss did not improve
 - 2s - loss: 0.1005 - acc: 0.9602 - val_loss: 0.3699 - val_acc: 0.8625
Epoch 113/120
Epoch 00113: val_loss did not improve
 - 2s - loss: 0.1181 - acc: 0.9562 - val_loss: 0.3666 - val_acc: 0.8750
Epoch 114/120
Epoch 00114: val_loss did not improve
 - 2s - loss: 0.1043 - acc: 0.9578 - val_loss: 0.3420 - val_acc: 0.8875
Epoch 11

In [15]:
with open('../features/cnn_4_aug_1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug_1_fold8.csv', index=False)

0.208893229909
         id  is_iceberg
0  5941774d    0.045843
1  4023181e    0.992780
2  b20200e4    0.061591
3  e7f018bb    0.996021
4  4371c8c3    0.563492


In [16]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [17]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.55090, saving model to best_m.h5
 - 7s - loss: 0.6876 - acc: 0.6279 - val_loss: 0.5509 - val_acc: 0.7259
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.4699 - acc: 0.7654 - val_loss: 0.7466 - val_acc: 0.6137
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4446 - acc: 0.7951 - val_loss: 0.6789 - val_acc: 0.6449
Epoch 4/120
Epoch 00004: val_loss improved from 0.55090 to 0.44434, saving model to best_m.h5
 - 4s - loss: 0.3945 - acc: 0.8159 - val_loss: 0.4443 - val_acc: 0.7539
Epoch 5/120
Epoch 00005: val_loss improved from 0.44434 to 0.40281, saving model to best_m.h5
 - 4s - loss: 0.3824 - acc: 0.8227 - val_loss: 0.4028 - val_acc: 0.7882
Epoch 6/120
Epoch 00006: val_loss improved from 0.40281 to 0.30031, saving model to best_m.h5
 - 3s - loss: 0.4057 - acc: 0.8194 - val_loss: 0.3003 - val_acc: 0.8692
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 3s - loss: 0.3688 - acc: 0.8263 - val_loss:

Epoch 65/120
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.2307 - acc: 0.9066 - val_loss: 0.2784 - val_acc: 0.8660
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.2367 - acc: 0.8992 - val_loss: 0.2419 - val_acc: 0.8972
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.2357 - acc: 0.9047 - val_loss: 0.2554 - val_acc: 0.8847
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.2401 - acc: 0.8964 - val_loss: 0.2710 - val_acc: 0.9003
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.2419 - acc: 0.9013 - val_loss: 0.2497 - val_acc: 0.8879
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 3s - loss: 0.2310 - acc: 0.9084 - val_loss: 0.2503 - val_acc: 0.8972
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 3s - loss: 0.2090 - acc: 0.9101 - val_loss: 0.2695 - val_acc: 0.8816
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 3s - loss: 0.2272 - acc: 0.9086 - val_loss: 0.2425 - val_acc: 0.8972
Epoch 73/120
Epo

Epoch 10/120
Epoch 00010: val_loss improved from 0.33582 to 0.29852, saving model to best_m.h5
 - 4s - loss: 0.3640 - acc: 0.8331 - val_loss: 0.2985 - val_acc: 0.8660
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 3s - loss: 0.3286 - acc: 0.8490 - val_loss: 0.4046 - val_acc: 0.8006
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 3s - loss: 0.3142 - acc: 0.8633 - val_loss: 0.3067 - val_acc: 0.8536
Epoch 13/120
Epoch 00013: val_loss improved from 0.29852 to 0.28014, saving model to best_m.h5
 - 4s - loss: 0.3123 - acc: 0.8680 - val_loss: 0.2801 - val_acc: 0.8723
Epoch 14/120
Epoch 00014: val_loss improved from 0.28014 to 0.27367, saving model to best_m.h5
 - 4s - loss: 0.2990 - acc: 0.8695 - val_loss: 0.2737 - val_acc: 0.8692
Epoch 15/120
Epoch 00015: val_loss improved from 0.27367 to 0.25979, saving model to best_m.h5
 - 4s - loss: 0.3000 - acc: 0.8633 - val_loss: 0.2598 - val_acc: 0.8816
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 3s - loss: 0.3002 - acc: 0.8675 

Epoch 00073: val_loss did not improve
 - 3s - loss: 0.2311 - acc: 0.8937 - val_loss: 0.2430 - val_acc: 0.8910
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 3s - loss: 0.2211 - acc: 0.9062 - val_loss: 0.2362 - val_acc: 0.8972
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 3s - loss: 0.2140 - acc: 0.9172 - val_loss: 0.2503 - val_acc: 0.8941
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 3s - loss: 0.2310 - acc: 0.9070 - val_loss: 0.2533 - val_acc: 0.8847
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 3s - loss: 0.2215 - acc: 0.9099 - val_loss: 0.2441 - val_acc: 0.9003
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 3s - loss: 0.2213 - acc: 0.9156 - val_loss: 0.2557 - val_acc: 0.8723
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 3s - loss: 0.2212 - acc: 0.9109 - val_loss: 0.2392 - val_acc: 0.8910
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 3s - loss: 0.2070 - acc: 0.9109 - val_loss: 0.2543 - val_acc: 0.8910
Epoch 81/120
Epoch 00081: val

Epoch 17/120
Epoch 00017: val_loss improved from 0.23515 to 0.23074, saving model to best_m.h5
 - 4s - loss: 0.3100 - acc: 0.8594 - val_loss: 0.2307 - val_acc: 0.9003
Epoch 18/120
Epoch 00018: val_loss improved from 0.23074 to 0.20358, saving model to best_m.h5
 - 4s - loss: 0.3173 - acc: 0.8662 - val_loss: 0.2036 - val_acc: 0.9221
Epoch 19/120
Epoch 00019: val_loss did not improve
 - 3s - loss: 0.3067 - acc: 0.8662 - val_loss: 0.5729 - val_acc: 0.7788
Epoch 20/120
Epoch 00020: val_loss did not improve
 - 3s - loss: 0.3039 - acc: 0.8664 - val_loss: 0.2253 - val_acc: 0.9128
Epoch 21/120
Epoch 00021: val_loss did not improve
 - 3s - loss: 0.3345 - acc: 0.8618 - val_loss: 0.2430 - val_acc: 0.9128
Epoch 22/120
Epoch 00022: val_loss did not improve
 - 3s - loss: 0.3024 - acc: 0.8677 - val_loss: 0.2141 - val_acc: 0.9159
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 3s - loss: 0.3011 - acc: 0.8662 - val_loss: 0.2598 - val_acc: 0.8847
Epoch 24/120
Epoch 00024: val_loss did not improve


Epoch 00081: val_loss did not improve
 - 3s - loss: 0.2339 - acc: 0.9008 - val_loss: 0.2075 - val_acc: 0.9159
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 3s - loss: 0.2294 - acc: 0.8976 - val_loss: 0.2111 - val_acc: 0.9190
Epoch 83/120
Epoch 00083: val_loss did not improve
 - 3s - loss: 0.2375 - acc: 0.8898 - val_loss: 0.2326 - val_acc: 0.9097
Epoch 84/120
Epoch 00084: val_loss did not improve
 - 3s - loss: 0.2450 - acc: 0.9052 - val_loss: 0.1989 - val_acc: 0.9252
Epoch 85/120
Epoch 00085: val_loss did not improve
 - 3s - loss: 0.2407 - acc: 0.9023 - val_loss: 0.2051 - val_acc: 0.9283
Epoch 86/120
Epoch 00086: val_loss did not improve
 - 3s - loss: 0.2156 - acc: 0.9078 - val_loss: 0.2140 - val_acc: 0.9221
Epoch 87/120
Epoch 00087: val_loss did not improve
 - 3s - loss: 0.2430 - acc: 0.9013 - val_loss: 0.2108 - val_acc: 0.9159
Epoch 88/120
Epoch 00088: val_loss did not improve
 - 3s - loss: 0.2256 - acc: 0.9045 - val_loss: 0.2187 - val_acc: 0.9128
Epoch 89/120
Epoch 00089: val

Epoch 24/120
Epoch 00024: val_loss did not improve
 - 3s - loss: 0.2713 - acc: 0.8810 - val_loss: 0.4020 - val_acc: 0.8380
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.3345 - acc: 0.8493 - val_loss: 0.2956 - val_acc: 0.8660
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.2852 - acc: 0.8898 - val_loss: 0.3478 - val_acc: 0.8629
Epoch 27/120
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2745 - acc: 0.8802 - val_loss: 0.2615 - val_acc: 0.9034
Epoch 28/120
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2996 - acc: 0.8779 - val_loss: 0.3395 - val_acc: 0.8598
Epoch 29/120
Epoch 00029: val_loss did not improve
 - 3s - loss: 0.2615 - acc: 0.8855 - val_loss: 0.3014 - val_acc: 0.8785
Epoch 30/120
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.3002 - acc: 0.8734 - val_loss: 0.4101 - val_acc: 0.8660
Epoch 31/120
Epoch 00031: val_loss did not improve
 - 3s - loss: 0.2700 - acc: 0.8945 - val_loss: 0.2832 - val_acc: 0.9034
Epoch 32/120
Epo

Epoch 90/120
Epoch 00090: val_loss did not improve
 - 3s - loss: 0.2063 - acc: 0.9125 - val_loss: 0.3199 - val_acc: 0.8847
Epoch 91/120
Epoch 00091: val_loss did not improve
 - 3s - loss: 0.2231 - acc: 0.9044 - val_loss: 0.2557 - val_acc: 0.9190
Epoch 92/120
Epoch 00092: val_loss did not improve
 - 3s - loss: 0.2096 - acc: 0.9086 - val_loss: 0.2557 - val_acc: 0.9252
Epoch 93/120
Epoch 00093: val_loss did not improve
 - 3s - loss: 0.1930 - acc: 0.9201 - val_loss: 0.2549 - val_acc: 0.9252
Epoch 94/120
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.2430 - acc: 0.8956 - val_loss: 0.2643 - val_acc: 0.9159
Epoch 95/120
Epoch 00095: val_loss did not improve
 - 3s - loss: 0.2047 - acc: 0.9211 - val_loss: 0.2637 - val_acc: 0.9190
Epoch 96/120
Epoch 00096: val_loss did not improve
 - 3s - loss: 0.2172 - acc: 0.9062 - val_loss: 0.2644 - val_acc: 0.9221
Epoch 97/120
Epoch 00097: val_loss did not improve
 - 3s - loss: 0.1990 - acc: 0.9109 - val_loss: 0.2466 - val_acc: 0.9159
Epoch 98/120
Epo

Epoch 33/120
Epoch 00033: val_loss did not improve
 - 4s - loss: 0.2353 - acc: 0.9000 - val_loss: 0.2729 - val_acc: 0.8750
Epoch 34/120
Epoch 00034: val_loss did not improve
 - 4s - loss: 0.2184 - acc: 0.9156 - val_loss: 0.2674 - val_acc: 0.8781
Epoch 35/120
Epoch 00035: val_loss did not improve
 - 4s - loss: 0.2446 - acc: 0.8930 - val_loss: 0.2654 - val_acc: 0.8781
Epoch 36/120
Epoch 00036: val_loss did not improve
 - 4s - loss: 0.2121 - acc: 0.9078 - val_loss: 0.2670 - val_acc: 0.8656
Epoch 37/120
Epoch 00037: val_loss improved from 0.26473 to 0.26268, saving model to best_m.h5
 - 4s - loss: 0.2641 - acc: 0.8820 - val_loss: 0.2627 - val_acc: 0.8750
Epoch 38/120
Epoch 00038: val_loss did not improve
 - 4s - loss: 0.2294 - acc: 0.8984 - val_loss: 0.2908 - val_acc: 0.8781
Epoch 39/120
Epoch 00039: val_loss did not improve
 - 4s - loss: 0.2282 - acc: 0.9055 - val_loss: 0.2951 - val_acc: 0.8812
Epoch 40/120
Epoch 00040: val_loss did not improve
 - 4s - loss: 0.2398 - acc: 0.8992 - val_los

Epoch 99/120
Epoch 00099: val_loss did not improve
 - 3s - loss: 0.2071 - acc: 0.9094 - val_loss: 0.2667 - val_acc: 0.8719
Epoch 100/120
Epoch 00100: val_loss did not improve
 - 3s - loss: 0.1947 - acc: 0.9187 - val_loss: 0.2793 - val_acc: 0.8781
Epoch 101/120
Epoch 00101: val_loss did not improve
 - 3s - loss: 0.2048 - acc: 0.9242 - val_loss: 0.2687 - val_acc: 0.8969
Epoch 102/120
Epoch 00102: val_loss did not improve
 - 3s - loss: 0.2116 - acc: 0.9156 - val_loss: 0.2680 - val_acc: 0.8812
Epoch 103/120
Epoch 00103: val_loss did not improve
 - 3s - loss: 0.2102 - acc: 0.9164 - val_loss: 0.2676 - val_acc: 0.8906
Epoch 104/120
Epoch 00104: val_loss did not improve
 - 3s - loss: 0.1942 - acc: 0.9234 - val_loss: 0.2767 - val_acc: 0.8906
Epoch 105/120
Epoch 00105: val_loss did not improve
 - 3s - loss: 0.1926 - acc: 0.9188 - val_loss: 0.2920 - val_acc: 0.8688
Epoch 106/120
Epoch 00106: val_loss did not improve
 - 3s - loss: 0.1851 - acc: 0.9226 - val_loss: 0.2653 - val_acc: 0.8812
Epoch 107

In [18]:
with open('../features/cnn_5_aug_1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_5_aug_1_fold8.csv', index=False)

0.220936560774
         id  is_iceberg
0  5941774d    0.214610
1  4023181e    0.895995
2  b20200e4    0.034479
3  e7f018bb    0.997006
4  4371c8c3    0.531016
